In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
def seconds_to_minutes(seconds: float, human_readable: bool = True):
    if not human_readable:
        return seconds / 60

    minutes = seconds // 60  # Integer division
    remaining_seconds = seconds % 60
    
    if remaining_seconds < 10:
        return f"{int(minutes)}.0{int(remaining_seconds)}"
    return f"{int(minutes)}.{int(remaining_seconds)}"

# Example usage
seconds = 125
minutes_hr = seconds_to_minutes(seconds)
minutes = seconds_to_minutes(seconds=seconds, human_readable=False)
print(f"{seconds} seconds is equal to {minutes_hr}")
print(minutes)

125 seconds is equal to 2.05
2.0833333333333335


In [23]:
import os
from pytubefix import YouTube
from modules.youtube import extract_youtube_video_id, get_video_metadata

#video_url = "https://youtu.be/JhU0yO43b6o?si=mfsiB2wajdY55US9"

video_url = "https://youtu.be/qdo5lMR1lX4?si=em_911Df-b8nc-VF"
video_id = extract_youtube_video_id(video_url)
video_meta = get_video_metadata(video_url)

audio_filename = video_meta['name']
audio_filepath = os.path.join("..", "transcript_audios", audio_filename)
yt = YouTube(url=f"https://www.youtube.com/watch?v={video_id}")
yt.streams

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="313" mime_type="video/webm" res="2160p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="401" mime_type="video/mp4" res="2160p" fps="30fps" vcodec="av01.0.12M.08" progressive="False" type="video">, <Stream: itag="271" mime_type="video/webm" res="1440p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="400" mime_type="video/mp4" res="1440p" fps="30fps" vcodec="av01.0.12M.08" progressive="False" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="av01.0.08M.08" progressive="False" type="video">, <Stream: itag="136" mi

In [24]:
download = yt.streams.get_audio_only().download(mp3=True, filename=audio_filepath)

In [25]:
download

'/Users/sudoleg/Projects/Personal/YouTubeGPT/scripts/../transcript_audios/I forced EVERYONE to use Linux.mp3'

## Generate transcript via API

In [ ]:
from openai import OpenAI

api_key = os.getenv("OPENAI_YTAI_API_KEY")
client = OpenAI(api_key=api_key)

audio_file= open(download, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="verbose_json",
  timestamp_granularities=['segment']
)

transcription_segments = [segment for segment in transcription.segments]
transcription_text = transcription.text

## Generate transcript locally

In [26]:
import whisper
import time

# Start the timer
start_time = time.time()

model = whisper.load_model("base")
transcription = model.transcribe(download)

# End the timer
end_time = time.time()

# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

transcription_segments = [segment for segment in transcription['segments']]
transcription_text = transcription['text']

/Users/sudoleg/Projects/Personal/YouTubeGPT/.venv/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load

Execution time: 86.10 seconds


In [30]:
for segment in transcription_segments:
    start = seconds_to_minutes(float(segment['start']))
    end = seconds_to_minutes(float(segment['end']))
    print(f"Timemstamp: {start} - {end}")
    print(segment['text'])
    print("---")

Timemstamp: 0.00 - 0.06
 No more windows, no more map books, I'm replacing everything with limits, and I'm gonna force all my employees to do this, and not just that.
---
Timemstamp: 0.06 - 0.08
 I'm gonna make them all use one computer.
---
Timemstamp: 0.08 - 0.10
 But first, are you guys cool with this?
---
Timemstamp: 0.10 - 0.11
 That's not you, sir!
---
Timemstamp: 0.11 - 0.12
 What's going on with that?
---
Timemstamp: 0.12 - 0.13
 That's not you, my friend!
---
Timemstamp: 0.13 - 0.14
 Why would you let me touch me?
---
Timemstamp: 0.14 - 0.16
 Alright, maybe it's gonna take some convincing.
---
Timemstamp: 0.16 - 0.17
 But let me show you what I'm doing.
---
Timemstamp: 0.17 - 0.18
 You see this computer right here?
---
Timemstamp: 0.18 - 0.20
 This is our insanely powerful render mission.
---
Timemstamp: 0.20 - 0.25
 We use the surrender and edit videos, live stream, or we have a big problem with the sucker.
---
Timemstamp: 0.25 - 0.26
 It's running Windows 11.
---
Timemstamp:

In [31]:
from modules.helpers import save_response_as_file

save_response_as_file(dir_name="../transcribed", filename=video_meta['name'], file_content=transcription_text)

In [32]:
from langchain_core.documents import Document
from modules.helpers import num_tokens_from_string

def create_documents_from_segments(transcription_segments, max_tokens=1024, model='gpt-4o-mini'):
    documents = []
    current_document = None
    current_token_count = 0
    
    for index, segment in enumerate(transcription_segments):
        s_text = segment['text']
        s_text_token_num = num_tokens_from_string(string=s_text, model=model)
        
        if current_document is None:
            # Initialize a new Document
            current_document = Document(
                page_content=s_text,
                metadata={
                    "start": segment['start'],
                    "end": segment['end']
                }
            )
            current_token_count = s_text_token_num
        else:
            # Check if adding this segment exceeds the max_tokens limit
            if current_token_count + s_text_token_num <= max_tokens:
                # Append text and update end metadata and token count
                current_document.page_content += s_text
                current_document.metadata["end"] = segment['end']
                current_token_count += s_text_token_num
            else:
                # Save the current document and start a new one
                documents.append(current_document)
                current_document = Document(
                    page_content=s_text,
                    metadata={
                        "start": segment['start'],
                        "end": segment['end']
                    }
                )
                current_token_count = s_text_token_num
    
    # Append the last document if it exists
    if current_document is not None:
        documents.append(current_document)
    
    return documents

documents: list[Document] = create_documents_from_segments(transcription_segments)

In [22]:
for i, d in enumerate(documents, start=1):
    print(f"Token num in {i}. document: {num_tokens_from_string(string=d.page_content, model="gpt-4o-mini")}.")
    #print(f"https://www.youtube.com/watch?v={video_id}?t={d.metadata['start']}")
    print(d.page_content)
    print(f"Timemstamp: {seconds_to_minutes(d.metadata['start'])} - {seconds_to_minutes(d.metadata['end'])}")
    print("---")

Token num in 1. document: 1014.
https://www.youtube.com/watch?v=G9boJpqiZTQ?t=0.0
 Busy weekend. Busy weekend. Talk to me. How did this happen? I mean, I think I think it's the worst and scariest answer possible, which is like a balance of Occam's razor and Halen's razor. The first is so obvious about, basically looking for the most simple solution and the evidence supporting that and not looking for anything further than that, which leads us into Halen's razor, which is never attribute something to malice when it can be explained with stupidity. And if I go back, I don't want to start on Saturday because I think that grossly misrepresent the problem. We have to go back three years and start looking at what has happened organizationally, what has happened with personnel selection, what has happened with training, what has happened with requirements, what's happened with the authorities, with authorizations, what's happening with the overall labor force within that specific organization